In [1]:
import pickle
import typing as tp

import numpy as np
import pandas as pd
import tree
import wandb

from gym import spaces
from gym_sts.envs import base
from sts_playground import utils
from sts_playground.autoencoder import tf_lib

2023-03-12 11:37:41.967163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/vlad/Repos/gym-sts/venv/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
2023-03-12 11:37:42.830713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-12 11:37:42.830763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
wandb.login()

wandb: Currently logged in as: vladfi. Use `wandb login --relogin` to force relogin


True

In [4]:
api = wandb.Api()

In [5]:
run = api.run('sts-ai/autoencoder/v1b1a44w')

In [6]:
top1 = run.summary['validation']['top1']

In [7]:
def to_dict(d) -> dict:
    if isinstance(d, wandb.old.summary.SummarySubDict):
        d = dict(d.items())
    
    if isinstance(d, (tp.Mapping, tp.Sequence)):
        return tree.map_structure(to_dict, d)
    
    return d

In [9]:
top1_dict = to_dict(top1)
top1_dict = tf_lib.convert_lists(top1_dict)

In [10]:
with open('../data/state_action_triples_100k.pkl', 'rb') as f:
    states = pickle.load(f)

states = states['state_before']

In [11]:
states = tf_lib.convert_lists(states)

In [12]:
obs_space = tf_lib.space_as_nest(base.OBSERVATION_SPACE)
tree.assert_same_structure(states, obs_space)

In [13]:
def one_hot_np(xs, n):
    flat_xs = xs.reshape([-1])
    flat_one_hot = np.eye(n)[flat_xs]
    new_shape = xs.shape + (n,)
    return flat_one_hot.reshape(new_shape)


def discrete_mean_accuracy(xs, n):
    xs = one_hot_np(xs, n)
    ps = np.mean(xs, 0)
    return np.max(ps, -1)
    # return scipy.stats.entropy(ps)
    

def mean_accuracy(space, xs):
    if isinstance(space, spaces.Discrete):
        assert len(xs.shape) == 1
        return discrete_mean_accuracy(xs, space.n)

    if isinstance(space, spaces.MultiBinary):
        assert xs.shape[-1] == space.n
        assert len(xs.shape) == 2
        ps = np.mean(xs, 0)
        acc = np.maximum(ps, 1.0 - ps)
        return np.mean(acc)

    if isinstance(space, spaces.MultiDiscrete):
        assert len(xs.shape) == 2
        num_components = space.nvec.shape[0]
        assert xs.shape[-1] == num_components

        n = space.nvec[0]
        if np.all(space.nvec == n):
            # If all the Discrete's are the same size, we can avoid the expensive
            # unstack and do a reshape instead.
            acc = discrete_mean_accuracy(xs, n)
            assert acc.shape == (num_components,)
            return np.mean(acc)
        
        raise NotImplementedError()

        losses = []
        logits = tf.split(x, tuple(space.nvec), axis=-1)
        for x_, y_ in zip(logits, tf.unstack(y, axis=-1)):
            losses.append(discrete_accuracy(x_, y_))
        loss = tf.stack(losses, axis=-1)
        return tf.reduce_mean(loss, axis=-1)


In [14]:
mean_acc = tree.map_structure(mean_accuracy, obs_space, states)

In [15]:
del states

In [16]:
def path_to_str(path: tuple) -> str:
    return '.'.join(map(str, path))

def df_from_structs(**structs):
    columns = {}
    
    structs = list(structs.items())
    first_struct = structs[0][1]
    
    paths, _ = zip(*tree.flatten_with_path(first_struct))
    columns['path'] = list(map(path_to_str, paths))

    for name, struct in structs:
        tree.assert_same_structure(struct, first_struct)
        columns[name] = tree.flatten(struct)

    return pd.DataFrame(columns)

In [17]:
df = df_from_structs(top1=top1_dict, baseline=mean_acc)
df['improvement'] = df['top1'] - df['baseline']

In [18]:
df.sort_values('top1').head(20)

,path,top1,baseline,improvement
678,combat_state.enemies.0.id,0.369900,0.380915,-0.011014
2357,persistent_state.map.boss,0.452691,0.451599,0.001092
2359,persistent_state.map.nodes,0.453787,0.451678,0.002109
2555,persistent_state.screen_type,0.623481,0.615430,0.008051
2334,combat_state.hand.3.is_playable,0.639648,0.648695,-0.009047
679,combat_state.enemies.0.intent,0.646701,0.388307,0.258395
1007,combat_state.enemies.1.id,0.702474,0.702528,-0.000054
1008,combat_state.enemies.1.intent,0.707465,0.706224,0.001241
2354,persistent_state.health.hp,0.715473,0.704879,0.010594
2329,combat_state.hand.1.card,0.737283,0.737389,-0.000106
